In [9]:
import os 
import pandas as pd
import numpy as np
import datetime as dt
# import ee
# import geemap as geemap
from IPython.display import HTML

import panel as pn
import param
import plotly.express as px
import plotly.graph_objects as go


from meteostat import Point, Daily

#pn.extension('plotly')

In [2]:
%load_ext google.cloud.bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gbif-challenge-deed5b20a659.json' ##change this

In [3]:
from google.cloud import bigquery

client = bigquery.Client()

In [4]:
# %%bigquery df_bq
# SELECT
#     *
# FROM `gbif-challenge.airflow_uploads.gbif_combined`
# WHERE DATE(eventDate) = "2022-04-04"
# ee.Initialize()

In [5]:
df = pd.read_csv('human_interference_sample.csv')
df.head()

,key,publishingCountry,basisOfRecord,scientificName,species,genericName,iucnRedListCategory,decimalLongitude,decimalLatitude,coordinateUncertaintyInMeters,eventDate,recordedBy,country,countryCode,date,is_invasive,avg_radiance,avg_deg_urban
0,3031987049,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-47.491164,-22.258903,30298.0,2021-01-13T05:07:00,registroscomunitariosderioclarosp,Brazil,BR,2021-01-13,False,1.463336,0.389394
1,3307456876,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-43.628159,-17.983999,587863.0,2021-06-17T13:21:01,Enrico A. R. Tosto,Brazil,BR,2021-06-17,False,0.234009,0.352468
2,3328022769,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-47.509019,-22.525994,30278.0,2021-07-09T17:33:00,mariosoares,Brazil,BR,2021-07-09,False,5.220234,0.820176
3,3335201455,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-46.934391,-21.306569,30396.0,2021-07-18T06:34:00,CRISTINA RAPPA,Brazil,BR,2021-07-18,False,1.010773,0.391316
4,3773393229,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-46.874809,-21.219875,30396.0,2021-07-18T19:34:00,ademircarosia,Brazil,BR,2021-07-18,False,0.943412,0.683220


In [20]:
#### all the filter widgets we need
latitude = pn.widgets.FloatInput(name='Latitude', value=5., step=1e-1, 
                                 start=min(df.decimalLatitude), end=max(df.decimalLatitude), width=150)
longitude = pn.widgets.FloatInput(name='Longitude', value=5., step=1e-1,
                                  start=min(df.decimalLongitude), end=min(df.decimalLongitude), width=150)
species = pn.widgets.Select(name='Species', options=list(df.species.unique()))

event_date = pn.widgets.DateRangeSlider(
    name='Date Range',
    start=min(pd.to_datetime(df['eventDate'])), end=max(pd.to_datetime(df['eventDate'])),
    value=(dt.datetime(2021, 1, 1), dt.datetime(2021, 12, 1)),
)

country = pn.widgets.Select(name='Country', options=list(df.country.unique()))

button = pn.widgets.Button(name='Update Plots', width= 200, button_type='success')


#### all our plot functions
### the world map view of occurrence data
def occ_plot(df=df):
    fig = px.scatter_geo(df, lat="decimalLatitude", lon='decimalLongitude', color='species')
    ## making the background transparent below
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    },
    margin=dict(t=0, b=0, l=0, r=0))
    return fig

## create the world scatter plot
plot_scatter = pn.pane.Plotly(occ_plot())


## function for creating the treemap to show specific point wise values 
def create_cards(df, query=None):
    if query:
        ## filter the dataframe to the selected point
        df = df.query(query)
        df = pd.melt(df, value_vars=['avg_radiance', 'is_invasive', 'avg_deg_urban'])
        fig = px.treemap(df, path=['variable'], values='value')
        fig.data[0].textinfo = 'label+value'

        level = 1 # write the number of the last level you have
        lvl_clr = "#5cb25d"
        font_clr = "black"

        fig.data[0]['marker']['colors'] =[lvl_clr for sector in fig.data[0]['ids'] if len(sector.split("/")) == level]
        fig.data[0]['textfont']['color'] = [font_clr  for sector in fig.data[0]['ids'] if len(sector.split("/")) == level]

        fig.data[0]['textfont']['size'] = 30
        return  fig


###instantiate the cards plot
plot_cards = pn.pane.Plotly(create_cards(df, f'decimalLatitude == -22.258903'), width=400, height=400)


## function for creating the treemap to show specific point wise values 
def create_trends(df, query=None):
    if query:
        ## filter the dataframe to the selected point
        df = df.query(query)
        
        print(df.eventDate)
        start = dt.datetime(2018, 1, 1)
        end = dt.datetime(2018, 12, 31)

        # Create Point for Vancouver, BC
#         pt = Point(df.loc[0,'decimalLatitude'], df.loc[0,'decimalLongitude'], 0)
        pt = Point(53.033981, -1.380991, 0)

        # Get daily data for 2018
        data = Daily(pt, start, end)
        data = data.fetch().reset_index(level=0)
        if data.shape[0] > 0:
            fig = px.line(data, x='time', y='tavg', template='plotly_white',
                          title='Climate Covariates')
            fig.update_layout(width=600)
            return  fig


###instantiate the cards plot
plot_trends = pn.pane.Plotly(create_trends(df, f'decimalLatitude == -22.258903'), width=400, height=400)

## display data, can delete
cols = ['decimalLatitude','decimalLongitude', 'eventDate', 'species']
display_data = pn.widgets.DataFrame(df[cols].head(20))

## dependent hidden function to run when a point is clicked in the plot_scatter
@pn.depends(plot_scatter.param.click_data, watch=True)
def _update_after_click_on_1(click_data):
    if click_data !=None:
        lat = click_data['points'][0]['lat']
        lon = click_data['points'][0]['lon']
        plot_cards.object = create_cards(df, f'decimalLatitude == {lat}')


def query(start="2022-04-04", end="2022-04-04"):
    sql = f"""
    SELECT
        *
    FROM `gbif-challenge.airflow_uploads.gbif_combined`
    WHERE DATE(eventDate) BETWEEN "{start}" AND "{end}"
    """
    bq = client.query(sql).to_dataframe() 
    return bq
    
def fetch_data(start="2022-04-04", end="2022-04-04"):
    global df, cols
    if button.clicks > 0:
        bq = query(event_date.value[0].strftime('%Y-%m-%d'), event_date.value[0].strftime('%Y-%m-%d')) ###change this to 1
        df = bq.copy()
        plot_scatter.object = occ_plot(df)
        plot_cards.object = create_cards(df, f"decimalLatitude == {df.loc[0,'decimalLatitude']}")
        plot_trends.object = create_trends(df, f"decimalLatitude == {df.loc[0,'decimalLatitude']}")

        plot_species.object = species_counts(df)
        display_data.value = df[cols]
        
### placeholder histogram plot of species counts
def species_counts(df=df):
    fig = px.histogram(df, y="species",color_discrete_sequence=['#5cb25d'])
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    return fig
    
plot_species = pn.pane.Plotly(species_counts())


button.on_click(fetch_data)

0    2021-01-13T05:07:00
Name: eventDate, dtype: object


Watcher(inst=Button(name='Update Plots', width=200), cls=<class 'panel.widgets.button.Button'>, fn=<function fetch_data at 0x7ff58b987550>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [34]:
## The main template to render, sidebar for text

template = pn.template.FastGridTemplate(
    title="🌏 GBIF Powered by Covariates",
#     sidebar=["We are interested bleh bleh bleh",
#              pn.pane.Markdown("## Select Filters"), latitude, longitude,
#              '', event_date, '', species, '', country, button],
    header = ['<a href="https://github.com/tekritesh/bio-conservation/tree/main">About</a>'],
    sidebar=["""We are interested bleh bleh bleh.\n We will hunt you down if you harm ANY flora or fauna."""],
    accent = '#5cb25d', sidebar_width = 280, background_color = '#f5f5f5',
    neutral_color = '#ffffff',
    corner_radius = 15
)

### specify which portion of the main page grid you want to place a plot in

template.main[:1, :] = pn.Row(pn.Column(latitude, longitude),
                              event_date,
                              pn.Column(species, country),
                              pn.Column('<br>',button))

template.main[1:4, :6]=pn.Tabs(('GBIF',pn.Column(plot_scatter)),
                              ('Radiance', pn.pane.HTML(HTML('map1.html'), width=600)), dynamic=True)

template.main[1:4, 6:12] = pn.Column(plot_cards)

template.main[4:7, :6]= pn.Column('### Species Counts', plot_species, height=400)
template.main[4:7, 6:12]= pn.Column(display_data, height=400)

template.main[7:10, :6] = pn.Column(plot_trends)



###color examples
##77cb
#'#faad55'
#'#f0a3bc'
#a18dd6

In [ ]:
## tells the terminal command to run the template variable as a dashboard

template.servable();

In [ ]:
####extra code

# center_lat = -3.497341
# center_lon = 38.642394

# zoomlevel=15

# get December image, we're using the "avg_rad" band
# viirs2020_01 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2022-01-01","2022-01-31").select('avg_rad').median()


# initialize our map
# map1 = geemap.Map()
# map1.addLayer(viirs2020_01, {}, "VIIRS-DNB")
#map1.to_html('map1.html')

# def get_map(lat=20.5936832, long=78.962883, zoom_start=5):
#     mm = fm.Map(location=[lat,long], zoom_start=zoom_start)
#     return mm

# map = get_map()


# @pn.depends(button.param.clicks, watch=True)
# def fetch_data(clicks,start="2022-04-04", end="2022-04-04"):
#     ##de = "2022-04-04"\
#     #end = event_date.value[0].strftime('%Y-%m-%d')
#     cols = ['decimalLatitude','decimalLongitude', 'eventDate', 'species']
#     if clicks > 0:
#         global df
#         sql = f"""
#         SELECT
#             *
#         FROM `gbif-challenge.airflow_uploads.gbif_combined`
#         WHERE DATE(eventDate) BETWEEN "{start}" AND "{end}"
#         """
#         bq = client.query(sql).to_dataframe() 
#         plot_scatter.object = occ_plot(bq)
#         plot_cards.object = create_cards(bq, f"decimalLatitude == {bq.loc[0,'decimalLatitude']}")
#         plot_species.object = species_counts(bq)
#         df = bq.copy()
#         return pn.widgets.DataFrame(df[cols].head(20))
    

In [19]:
# # from datetime import datetime

# # from meteostat import Point, Daily

# # Set time period
# start = dt.datetime(2018, 1, 1)
# end = dt.datetime(2018, 12, 31)

# # Create Point for Vancouver, BC
# vancouver = Point(53.033981, -47.491164, 0)

# # Get daily data for 2018
# data = Daily(vancouver, start, end)
# data = data.fetch().reset_index(level=0)
# fig = px.line(data, x='time', y='tavg',
#               title='Climate Covariates')

# fig.show()

/Users/advikabattini/anaconda/envs/gbif/lib/python3.8/site-packages/meteostat/core/warn.py:32: Warning:

Cannot load daily/EGXN0.csv.gz from https://bulk.meteostat.net/v2/

